## Deliverable 2. Create a Customer Travel Destinations Map.

In [9]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from Config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,La Palma,US,33.8464,-118.0467,64.99,63,90,3.44,overcast clouds,0.0,0.0
1,1,Hastings,GB,50.8552,0.5729,51.80,62,13,16.11,few clouds,0.0,0.0
2,2,Hambantota,LK,6.1241,81.1185,80.44,81,60,4.00,broken clouds,0.0,0.0
3,3,Axim,GH,4.8699,-2.2405,82.87,79,94,8.68,light rain,0.3,0.0
4,4,Petropavlovsk-Kamchatskiy,RU,53.0452,158.6483,33.80,100,90,4.47,overcast clouds,0.0,0.0


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,La Palma,US,33.8464,-118.0467,64.99,63,90,3.44,overcast clouds,0.0,0.0
2,2,Hambantota,LK,6.1241,81.1185,80.44,81,60,4.00,broken clouds,0.0,0.0
3,3,Axim,GH,4.8699,-2.2405,82.87,79,94,8.68,light rain,0.3,0.0
7,7,Hithadhoo,MV,-0.6000,73.0833,83.50,70,46,7.81,scattered clouds,0.0,0.0
10,10,Atuona,PF,-9.8000,-139.0333,80.65,73,25,21.23,scattered clouds,0.0,0.0
11,11,Poum,NC,-20.2333,164.0167,78.28,81,97,14.20,overcast clouds,0.0,0.0
12,12,Ampanihy,MG,-24.7000,44.7500,71.53,71,44,2.98,scattered clouds,0.0,0.0
13,13,Hilo,US,19.7297,-155.0900,69.80,88,90,3.44,moderate rain,3.3,0.0
16,16,Isangel,VU,-19.5500,169.2667,77.25,80,86,9.86,overcast clouds,0.0,0.0
17,17,Puerto Ayora,EC,-0.7393,-90.3518,80.60,74,75,16.11,broken clouds,0.0,0.0


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             1906
City                   1906
Country                1881
Lat                    1906
Lng                    1906
Max Temp               1906
Humidity               1906
Cloudiness             1906
Wind Speed             1906
Current Description    1906
Rain (inches)           382
Snow (inches)           382
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,La Palma,US,64.99,overcast clouds,33.8464,-118.0467,
2,Hambantota,LK,80.44,broken clouds,6.1241,81.1185,
3,Axim,GH,82.87,light rain,4.8699,-2.2405,
7,Hithadhoo,MV,83.50,scattered clouds,-0.6000,73.0833,
10,Atuona,PF,80.65,scattered clouds,-9.8000,-139.0333,
11,Poum,NC,78.28,overcast clouds,-20.2333,164.0167,
12,Ampanihy,MG,71.53,scattered clouds,-24.7000,44.7500,
13,Hilo,US,69.80,moderate rain,19.7297,-155.0900,
16,Isangel,VU,77.25,overcast clouds,-19.5500,169.2667,
17,Puerto Ayora,EC,80.60,broken clouds,-0.7393,-90.3518,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [25]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,La Palma,US,64.99,overcast clouds,33.8464,-118.0467,La Quinta Inn & Suites by Wyndham Buena Park
2,Hambantota,LK,80.44,broken clouds,6.1241,81.1185,Bungalow 63
3,Axim,GH,82.87,light rain,4.8699,-2.2405,AXIM BEACH RESORT AND SPA
7,Hithadhoo,MV,83.50,scattered clouds,-0.6000,73.0833,Scoop Guest House
10,Atuona,PF,80.65,scattered clouds,-9.8000,-139.0333,Villa Enata
...,...,...,...,...,...,...,...
3359,Vallenar,CL,76.08,clear sky,-28.5708,-70.7581,Humacao Bed & Breakfast
3361,Cowansville,CA,64.40,broken clouds,45.2001,-72.7491,L'Auberge des Carrefours
3362,Gushikawa,JP,66.20,moderate rain,26.3544,127.8686,Kirari Guest House
3363,Kieta,PG,78.87,broken clouds,-6.2167,155.6333,Uruna Bay Retreat


In [30]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center = (30.0,31.0), zoom_level = 1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))